# Exercici 4.5 Levenshtein


In [1]:
def dna(patro, fitxer = 'HUMAN-DNA.txt'):
    #Carreguem el fitxer
    with open('HUMAN-DNA.txt', 'r') as f:
        text = f.read()
        lineas = text.split('\n')
    
    #Calculem la llargada del patro
    patro_length = len(patro) + 1

    #Proposem una distancia maxima
    distancia_final = 2*patro_length

    #Index de linia
    index_linia = 0

    #Posicio inicial, final
    posicio = ()

    #Seqüència operacions
    operacions = []
        
    #Per a cada linia
    for linea in lineas:
        #Apliquem levenshtein
        distancia, pos, operacions_final = levenshtein(patro, linea)

        #Guardem la línia a minima distancia d'edicio, actualitzem la tupla de posició, guardem la linea i actualitzem les operacions realitzades. 
        if distancia < distancia_final:
            distancia_final = distancia
            posicio  = pos
            operacions = operacions_final
            linia = index_linia
        
        #Sumem un al index de linia
        index_linia += 1

    return linia, posicio, distancia_final, operacions
        
def levenshtein(patro, text, dlt = 2, insr = 2, subs = 1):
    #Mirem la llargada de la linia
    text_length = len(text) + 1

    #Calculem la llargada del patro
    patro_length = len(patro) + 1
    
    #Inicialitzem la matriu
    distance_matrix = [[0] * text_length for x in range(patro_length)]
    operation_matrix = [["D"] * text_length for j in range(patro_length)]
    operation_matrix[0][0] = 'S'
    sequencia_operacions = []

    for i in range(patro_length):
        distance_matrix[i][0] = i
    for i in range(text_length):
        operation_matrix[0][i] = 'I'
    
    #Omplim la matriu de distàncies:    
    for i in range(1, patro_length):
        for j in range(1, text_length):
            correspondencia = 0
            if patro[i-1] != text[j-1]:
                correspondencia = subs
            deletion = distance_matrix[i-1][j] + dlt
            insertion = distance_matrix[i][j-1] + insr
            substitution = distance_matrix[i-1][j-1] + correspondencia
            
            distance_matrix[i][j] = min(insertion, deletion, substitution)

            #Omplim la matriu d'operacions
            if min(insertion, deletion, substitution) == substitution and correspondencia == 0:
                operation_matrix[i][j] = 'C'
            elif min(insertion, deletion, substitution) == substitution:
                operation_matrix[i][j] = 'S'
            elif min(insertion, deletion, substitution) == insertion:
                operation_matrix[i][j] = 'I'
            
          
    #Calculem la distància final i la posició final.  
    distancia_minima = min(distance_matrix[patro_length-1])
    posicio_final = distance_matrix[patro_length-1].index(distancia_minima)

    #Càlcul de la posició on comença la subseqüència més semblant i busqueda de les operacions realitzades.
    x = len(patro)
    y = posicio_final
    while x >= 1 and y >= 1:
        if operation_matrix[x][y] == 'C':
            x -= 1
            y -= 1
            sequencia_operacions.append('C')
        elif operation_matrix[x][y] == 'S':
            x -= 1
            y -= 1
            sequencia_operacions.append('S')
        elif operation_matrix[x][y] == 'I':
            y -= 1
            sequencia_operacions.append('I')
        else:
            x -= 1
            sequencia_operacions.append('D')
    posicio_inicial = y
    sequencia_operacions = sequencia_operacions[::-1]
    
    return distancia_minima, (posicio_inicial, posicio_final), sequencia_operacions       

In [ ]:
assert dna("CTGGTACCAGCTGTATTAGC") == (728, (11, 31), 6, ['C', 'C', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'S', 'C', 'S', 'S', 'S', 'C', 'C', 'S', 'C', 'C', 'C'])
assert dna("TCGTCATAAACCGCTGTGCC") == (212, (12, 32), 7, ['S', 'C', 'S', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'S', 'S', 'C', 'C', 'S', 'S', 'C', 'S'])
assert dna("TATACAAACGGAGTAGCTGT") == (285, (5, 25), 6, ['C', 'C', 'C', 'C', 'S', 'C', 'S', 'C', 'C', 'S', 'C', 'S', 'S', 'C', 'C', 'C', 'S', 'C', 'C', 'C'])
assert dna("AGGCGTAAGTCTTACGTATA") == (5, (41, 61), 7, ['C', 'S', 'C', 'S', 'S', 'C', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'S', 'S', 'C', 'S', 'C', 'C', 'C'])
assert dna("AACGGCATAGCCTGCAAGAG") == (433, (41, 61), 5, ['C', 'C', 'S', 'C', 'C', 'C', 'C', 'S', 'C', 'S', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'S'])
assert dna("GTGCGTCCACCCTTAATACA") == (196, (41, 61), 6, ['C', 'C', 'C', 'S', 'S', 'C', 'S', 'C', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'S', 'C', 'C', 'S', 'C'])
assert dna("CCCTAAAACCAAAAGTGTTG") == (199, (30, 49), 6, ['S', 'C', 'C', 'C', 'S', 'C', 'C', 'C', 'S', 'S', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'D', 'C', 'C'])
assert dna("GTCAGCACCGGGATCTGTAT") == (240, (26, 46), 7, ['C', 'S', 'C', 'S', 'S', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'C', 'S', 'C', 'S', 'C', 'C', 'S', 'C'])
assert dna("GAGCCCCGACGTTTTAACGA") == (68, (6, 26), 7, ['S', 'C', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'C', 'S', 'C', 'S', 'C', 'C', 'C', 'S', 'S', 'S', 'C'])
assert dna("CACACCTTTCAGTACGTGAA") == (40, (14, 32), 7, ['C', 'C', 'C', 'D', 'C', 'S', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'D', 'S', 'C', 'C'])
assert dna("CCTCGTAGACAGTACCGAAT") == (448, (30, 50), 6, ['C', 'S', 'C', 'C', 'S', 'C', 'C', 'S', 'C', 'C', 'C', 'S', 'C', 'C', 'C', 'C', 'S', 'S', 'C', 'C'])
assert dna("CGACCAAAGAGCCTGTATCT") == (320, (35, 55), 7, ['S', 'S', 'C', 'S', 'S', 'C', 'C', 'C', 'C', 'S', 'C', 'S', 'C', 'C', 'S', 'C', 'C', 'C', 'C', 'C'])
assert dna("CGTGGTGTCCATACCCTAGC") == (835, (24, 43), 6, ['C', 'S', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'C', 'C', 'C', 'S', 'C', 'C', 'C', 'D', 'C', 'S', 'C'])
assert dna("GTGATAGACCTTTTAAGCTG") == (409, (18, 37), 6, ['S', 'C', 'C', 'C', 'C', 'C', 'D', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'C', 'C', 'S', 'S', 'C', 'C'])
assert dna("TAAGTCTTTGGTCACCCCCG") == (19, (10, 29), 7, ['C', 'S', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'C', 'C', 'D', 'C', 'C', 'C', 'C', 'S', 'C', 'S', 'S'])
assert dna("GACACACACTTGGATCTTCG") == (565, (16, 36), 6, ['C', 'S', 'C', 'C', 'C', 'S', 'S', 'C', 'C', 'C', 'C', 'C', 'S', 'S', 'S', 'C', 'C', 'C', 'C', 'C'])
assert dna("TCATAGCCTCGGGATAGTAT") == (306, (27, 46), 7, ['S', 'S', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'S', 'S', 'C', 'C', 'C', 'D', 'C', 'C', 'S'])
assert dna("CTGGACGTTCATACATAGAC") == (28, (21, 41), 7, ['C', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'C', 'S', 'S', 'S', 'C', 'S', 'C', 'S', 'C', 'C', 'C', 'S'])
assert dna("ACGTTTTACCCCAAAGCCCG") == (753, (4, 24), 7, ['C', 'S', 'S', 'S', 'S', 'C', 'C', 'C', 'C', 'S', 'C', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'S', 'C'])
assert dna("CGGGTAGAAATCTCCGCTTG") == (361, (30, 50), 6, ['S', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'S', 'S', 'C', 'C', 'S', 'C', 'C', 'S', 'S', 'C', 'C', 'C'])
print("All tests passed!")

All tests passed!
